# Bot for trading RVNUSDT with upwards trend and Moving average

### Installs

In [ ]:
#%pip install python-binance 
#%pip install numpy
#%pip install pandas

### imports and definition of the coin

In [1]:
import config
from binance.client import Client
from binance.enums import *
import time
import numpy as np

cliente = Client(config.API_KEY, config.API_SECRET, tld='com')

simbolo = 'RVNUSDT'   #Name of specific Symbol we are going to trade
quantityOrder = 500 # Quantity of RVN I am going to buy per operation. Around 20$...

### Functions

In [2]:
### THIS BOT IS GOING TO OPERATE WITH TRENDS AND LONG TERM MOBILE LINES

def trend():
    x = []
    y = []
    sum = 0
    ma48_i = 0

    resp = False

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_15MINUTE, "18 hour ago UTC")

    if (len(klines) != 72):
        
        return False
    for i in range (1,72):  # From 24 to 60, we are going to bring 36 Candlesticks of 15mins which will do the average of 9h

        for j in range (i-72,i): 
          sum = sum + float(klines[i][4])  # 4 value makes reference to the closing price of a candlestick
        ma48_i = '{:.5f}'.format(sum /72)  #.5 is the quantity of decimals we are going to use. We are giving format to the value
        sum = 0
        x.append(i)
        y.append(float(ma48_i))

    model = np.polyfit(x,y,1)
    if (model[0]>0):
        resp = True
    

    return resp
    


def _ma48_():

    ma48_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_15MINUTE, "12 hour ago UTC")

    if(len(klines)==48):
        for i in range (0,48):
            sum = sum +float(klines[i][4]) # 4 value makes reference to the closing price of a candlestick
        
        ma48_local = sum / 48

    return ma48_local

### *** Loop operation for continuous trading ***

In [19]:
while 1:
    orders = cliente.get_open_orders(symbol=simbolo)
    print("Actual Open Orders: ") 
    print(orders)

    if(len(orders) !=0):   # If there is open orders the bot is not going to open a new buy order
        print("Open orders already exist, no more buys")
        time.sleep(10)
        continue

    # Bring actual price of the cryptocoin

    list_of_tickers = cliente.get_all_tickers()
    for tick_2 in list_of_tickers:
        if tick_2['symbol'] == simbolo:
            symbolPrice = float(tick_2['price'])    # Price
    

    ma48 = _ma48_()
    if (ma48 == 0): continue

    print("---------" + simbolo + "-----------")
    print(" Actual price of MA48 " + str('{:.8f}'.format(ma48)))     #The .8 is for the decimals Binance's API is going to return us
    print(" Crypto's actual price  "+ str('{:.8f}'.format(symbolPrice)))
    print(" Price to buy at "+ str('{:.8f}'.format(ma48*0.995)))    # 0.995 represents security value to execute an operation

    if (not trend()): 
        print ("Downwards trend, no buy orders are going to be executed")

        time.sleep(10)
        continue
    else:
        print("Upwards trennd, buy if no open orders already")

    if(symbolPrice > ma48*0.995):
        print("***  BUYING  ***")

    order = cliente.order_market_buy(
        #API =   local
            symbol = simbolo,
            quantity = quantityOrder
            
        )
    time.sleep(5)

    # Put OCO order (One cancells other)

    info = cliente.get_symbol_info(simbolo)
    
    #print(info['filters'][2]['minQty'])

    ordenOCO = cliente.create_oco_order(
            symbol = simbolo,
            side = SIDE_SELL,
            stopLimitPrice = str('{:.5f}'.format(symbolPrice*0.985)),
            stopLimitTimeInForce = TIME_IN_FORCE_GTC,
            quantity = quantityOrder*0.999,   # We are going to put 0.999 as Binance charges a fee for the operation, so if there is tight funds Binance will return 'Insuficient funds'
            stopPrice = str('{:.5f}'.format(symbolPrice*0.99)),
            price = str('{:.5f}'.format(symbolPrice*1.01)),
            )
    
    time.sleep(20) #mando el robot a dormir porque EN TEORIA abrio un orden, dejamos que el mercado opere.

    # ERRORS

    # Signature for this request is not valid. API o secret are wrong. 
    # Invalid symbol: control that the crypto symbol of the pair is correctly written. 
    # PRICE_FILTER. decimals in OCO order might be wrong...check decimals that are accepted for that pair in binance
    # Min_NOTIONAL: depends on minimum of lot size that we can buy or sell...for example, if minimum is 10USDT, we must try and buy a little over that quantity..
    # The relationship of the prices for the orders is not correct. 


Actual Open Orders: 
[{'symbol': 'RVNUSDT', 'orderId': 640612406, 'orderListId': 79466124, 'clientOrderId': 'KVtOQeyMQGUm2LrlqzFodd', 'price': '0.02174000', 'origQty': '499.50000000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'STOP_LOSS_LIMIT', 'side': 'SELL', 'stopPrice': '0.02185000', 'icebergQty': '0.00000000', 'time': 1671012210149, 'updateTime': 1671012210149, 'isWorking': False, 'origQuoteOrderQty': '0.00000000'}, {'symbol': 'RVNUSDT', 'orderId': 640612407, 'orderListId': 79466124, 'clientOrderId': 'mWRnbRmPEJbrQ7RsYz3g4L', 'price': '0.02229000', 'origQty': '499.50000000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT_MAKER', 'side': 'SELL', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': 1671012210149, 'updateTime': 1671012210149, 'isWorking': True, 'origQuoteOrderQty': '0.00000000'}]
Open orders already exist, no more bu

KeyboardInterrupt: 